In [6]:
import numpy as np
import scipy.special
import scipy.optimize
import sklearn.datasets
class LinearLogisticRegression:

    def __init__(self, lbd, prior_weighted=False, prior=0.5):
        self.lbd = lbd
        self.prior_weighted = prior_weighted
        self.prior = prior


    def __compute_zi(self, ci):
        return 2 * ci - 1

    def __logreg_obj(self, v):
        w, b = v[0:-1], v[-1]
        z = 2 * self.Ltrain - 1
        exp = (np.dot(w.T, self.Dtrain) + b)
        reg = 0.5 * self.lbd * np.linalg.norm(w) ** 2
        avg_risk = (np.logaddexp(0, -exp * z)).mean()
        return reg + avg_risk

    def __logreg_obj_prior_weighted(self, v):
        w, b = v[0:-1], v[-1]
        z = 2 * self.Ltrain - 1
        reg = 0.5 * self.lbd * np.linalg.norm(w) ** 2
        exp = (np.dot(w.T, self.Dtrain) + b)
        avg_risk_0 = np.logaddexp(0, -exp[self.Ltrain == 0] * z[self.Ltrain == 0]).mean()*(1-self.prior)
        avg_risk_1 = np.logaddexp(0, -exp[self.Ltrain == 1] * z[self.Ltrain == 1]).mean() * self.prior
        return reg + avg_risk_0 + avg_risk_1
    def train(self, Dtrain, Ltrain):
        self.Dtrain = Dtrain
        self.Ltrain = Ltrain
        self.F = Dtrain.shape[0]  # dimensionality of features space
        self.K = len(set(Ltrain))  # number of classes
        self.N = Dtrain.shape[1]
        obj_function = self.__logreg_obj if self.prior_weighted is False else self.__logreg_obj_prior_weighted
        self.x, f, d = scipy.optimize.fmin_l_bfgs_b(func=obj_function,
                                                    x0=np.zeros(self.Dtrain.shape[0] + 1),
                                                    approx_grad=True,
                                                    iprint=0)

        """                                            
        print('Point of minimum: %s' % (self.x))
        print('Value of the minimum: %s' % (f))
        print('Number of iterations: %s' % (d['funcalls']))
        """
        return self

    def predict(self, Dtest, labels=True):
        w, b = self.x[0:-1], self.x[-1]
        S = np.zeros((Dtest.shape[1]))
        for i in range(Dtest.shape[1]):
            xi = Dtest[:, i:i + 1]
            s = np.dot(w.T, xi) + b
            S[i] = s
        if labels:
            LP = S > 0
            return LP
        else:
            return S

class QuadraticLogisticRegression:
    def __init__(self):
        pass

    def __compute_zi(self, ci):
        return 2 * ci - 1

    def __logreg_obj(self, v):  # still works if DTR is one sample only? yes but it must be of shape (4,1)
        w, b = v[0:-1], v[-1]
        J = self.lbd / 2 * (np.linalg.norm(w) ** 2)
        summary = 0
        for i in range(self.N):
            xi = self.Dtrain[:, i:i + 1]
            ci = self.Ltrain[i]
            zi = self.__compute_zi(ci)
            summary += np.logaddexp(0, -zi * (np.dot(w.T, xi) + b))
        J += (1 / self.N) * summary
        return J

    def train(self, Dtrain, Ltrain, lbd, maxiter):
        self.Dtrain = Dtrain
        self.Ltrain = Ltrain
        self.lbd = lbd
        self.maxiter = maxiter
        self.F = Dtrain.shape[0]  # dimensionality of features space
        self.K = len(set(Ltrain))  # number of classes
        self.N = Dtrain.shape[1]
        DTR_ext = numpy.hstack([self.__expandFeatures(self.Dtrain[:, i]) for i in range(self.Dtrain.shape[1])])
        self.x, f, d = scipy.optimize.fmin_l_bfgs_b(func=self.__logreg_obj, 
                                                    x0=numpy.zeros(DTR_ext.shape[0] + 1), 
                                                    approx_grad = True, 
                                                    factr=1.0)
        print('Point of minimum: %s' % (self.x))
        print('Value of the minimum: %s' % (f))
        print('Number of iterations: %s' % (d['funcalls']))
        return self    
    
    def __expandFeatures(x):
        x = utils.mcol(x)
        expX = utils.mcol(numpy.dot(x, x.T))
        return numpy.vstack([expX, x])

    def predict(self, Dtest):
        DTE_ext = numpy.hstack([self.__expandFeatures(Dtest[:, i]) for i in range(Dtest.shape[1])])
        w, b = utils.mcol(self.x[0:-1]), self.x[-1]
        scores = numpy.dot(w.T, DTE_ext) + b
        return scores[0, :]

In [8]:
def load_iris_binary():
    D, L = sklearn.datasets.load_iris()['data'].T, sklearn.datasets.load_iris()['target']
    D = D[:, L != 0] # remove setosa from D
    L = L[L!=0] # remove setosa from L
    L[L==2] = 0 # We assign label 0 to virginica (was label 2)
    return D, L
def split_db_2to1(D, L, seed=0):
    nTrain = int(D.shape[1]*2.0/3.0) # 2/3 of the dataset D are used for training, 1/3 for validation
    np.random.seed(seed)
    idx = np.random.permutation(D.shape[1]) # take a random array of 150 elements, each element is 0<x<=149 (np.arange(150))
    idxTrain = idx[0:nTrain] # first 100 are indices of training samples
    idxTest = idx[nTrain:] # remaining 50 are indices of validation samples
    DTR = D[:, idxTrain] # D for training
    DTE = D[:, idxTest] # D for validation
    LTR = L[idxTrain] # L for training
    LTE = L[idxTest] # L for validation
    return (DTR, LTR), (DTE, LTE)

DT, LT = load_iris_binary()
(DTR, LTR), (DTE, LTE) = split_db_2to1(DT, LT)

In [17]:
lbd = 10**-3
maxiter=100
model = LinearLogisticRegression(lbd, prior_weighted=True, prior=0.5)
labs = model.train(DTR, LTR).predict(DTE, labels=True)
print('Error rate: ', (1 - (sum(labs == LTE) / len(LTE))) * 100)

Error rate:  8.823529411764708


In [ ]:
lbd = 10**-3
maxiter=100
model = QuadraticLogisticRegression()
labs = model.train(DTR, LTR, lbd, maxiter).predict(DTE, labels=True)
print('Error rate: ', 1 - (sum(labs == LTE) / len(LTE)))

In [18]:
np.linspace(10**-6, 10**5, 100)

array([1.00000000e-06, 1.01010101e+03, 2.02020202e+03, 3.03030303e+03,
       4.04040404e+03, 5.05050505e+03, 6.06060606e+03, 7.07070707e+03,
       8.08080808e+03, 9.09090909e+03, 1.01010101e+04, 1.11111111e+04,
       1.21212121e+04, 1.31313131e+04, 1.41414141e+04, 1.51515152e+04,
       1.61616162e+04, 1.71717172e+04, 1.81818182e+04, 1.91919192e+04,
       2.02020202e+04, 2.12121212e+04, 2.22222222e+04, 2.32323232e+04,
       2.42424242e+04, 2.52525253e+04, 2.62626263e+04, 2.72727273e+04,
       2.82828283e+04, 2.92929293e+04, 3.03030303e+04, 3.13131313e+04,
       3.23232323e+04, 3.33333333e+04, 3.43434343e+04, 3.53535354e+04,
       3.63636364e+04, 3.73737374e+04, 3.83838384e+04, 3.93939394e+04,
       4.04040404e+04, 4.14141414e+04, 4.24242424e+04, 4.34343434e+04,
       4.44444444e+04, 4.54545455e+04, 4.64646465e+04, 4.74747475e+04,
       4.84848485e+04, 4.94949495e+04, 5.05050505e+04, 5.15151515e+04,
       5.25252525e+04, 5.35353535e+04, 5.45454545e+04, 5.55555556e+04,
      

In [27]:
def trainLinearSVM(DTR, LTR, K, C, DTE, p = 0):
    Z = LTR * 2.0 - 1.0
    X_hat = np.vstack([DTR, K * np.ones((1, DTR.shape[1]))])
    G = np.dot(X_hat.T, X_hat)
    H_hat = Z.reshape(-1,1) * Z.reshape(1,-1) * G
    empP = (LTR == 1).sum()/len(LTR)
    alphaBounds = np.array([(0, C)] * LTR.shape[0])
    
    if p != 0:
        alphaBounds[LTR == 1] = (0, C*p/empP)
        alphaBounds[LTR == 0] = (0, C*(1-p)/(1-empP))
    
    def computeDualLoss(alpha):   
        return 0.5 * np.dot(np.dot(alpha.reshape(1,-1), H_hat), alpha) - alpha.sum(), np.dot(H_hat, alpha) - 1
        
    def computePrimalFromDual(alpha):
        w_hat = np.dot(alpha, (Z * X_hat).T)
        w = w_hat[:-1]
        b = w_hat[-1::]                
        return w_hat, w, b
    
    def computeSVMScore(w, b):
        return np.dot(w.T, DTE) + b*K
    
    alphaStar, x, y = scipy.optimize.fmin_l_bfgs_b(
        computeDualLoss, 
        np.zeros(DTR.shape[1]), 
        bounds = alphaBounds, 
        factr=1.0,
        maxfun=100000,
        maxiter=100000)

    w_hat, w, b = computePrimalFromDual(alphaStar)    
    score = computeSVMScore(w, b)
    # print(f'Elapsed {time.time() - t} seconds')

    return score

In [29]:
trainLinearSVM(DTR, LTR, 1, 0.1, DTE, p=0.1)

array([-0.65013479, -1.09932291, -0.60630051, -1.14728007, -0.65037293,
       -0.51364522, -0.72628327, -1.10755269, -1.23297549, -1.01174305,
       -0.98934839, -1.32817439, -1.39589771, -1.01111484, -0.74778862,
       -0.78553736, -1.24814348, -0.74448234, -0.7347432 , -1.20625326,
       -1.21583497, -0.63719537, -0.99415472, -1.26680952, -1.12868295,
       -0.833613  , -0.67290205, -0.97786307, -0.74668182, -1.07743798,
       -1.38828666, -1.25207186, -0.73230558, -0.76385711])